In [1]:
import pymongo
import pandas as pd
import itertools
import csv
import tqdm

In [2]:
%run filepaths.ipynb

In [3]:
client = pymongo.MongoClient()
db = client['uni']

In [4]:
def get_graduates(vpo1_collection):
    return vpo1_collection.aggregate(
        pipeline=[
    {
        '$match': {
            'section': '2.1.3',
            'region': { '$ne': 'Российская Федерация' }
        }
    }, {
        '$project': {
            'columns': 0
        }
    }, {
        '$unwind': {
            'path': '$rows', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$match': {
            'rows.values.2.colname': 'Код классификатора', 
            'rows.values.2.value': 1
        }
    }, {
        '$unwind': {
            'path': '$rows.values', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$match': {
            'rows.values.colname': {
                '$regex': 'Выпуск фактический.*',
                '$not': {'$regex': '.*инвалиды.*'}
            }
        }
    }, {
        '$project': {
            'region': 1, 
            'funded_by': 1, 
            'time_involvement': 1, 
            # 'section': 1, 
            # 'section_name': 1, 
            'colname': '$rows.values.colname', 
            'group': '$rows.group', 
            'program': '$rows.name', 
            'value': '$rows.values.value'
        }
    }
])

In [5]:
fieldnames = list(next(get_graduates(db.vpo1)).keys())
fieldnames.remove('_id')
# fieldnames.remove('group')
fieldnames

['region',
 'funded_by',
 'time_involvement',
 'colname',
 'group',
 'program',
 'value']

In [6]:
with open(GRADUATES_FILE, 'w', newline='') as f:
    f.truncate(0)
    writer = csv.DictWriter(f, fieldnames=fieldnames, dialect='unix')
    writer.writeheader()
    for record in tqdm.tqdm(get_graduates(db.vpo1)):
        # record['program'] = '{}. {}'.format(record['group'], record['program'])
        for k in list(record.keys()):
            if k not in fieldnames:
                del record[k]
        writer.writerow(record)

18922it [00:03, 5813.78it/s]


In [7]:
df = pd.read_csv(GRADUATES_FILE)
df

,region,funded_by,time_involvement,colname,group,program,value
0,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Математика,5.0
1,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Прикладная математика и информатика,9.0
2,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Математическое обеспечение и администрирование...,20.0
3,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Физика,0.0
4,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Радиофизика,10.0
5,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Химия,26.0
6,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,География,17.0
7,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Экология и природопользование,24.0
8,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Биология,16.0
9,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Строительство,61.0


In [8]:
df[df['program'] == 'Математика'].head()

,region,funded_by,time_involvement,colname,group,program,value
0,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Математика,5.0
70,Калининградская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,3,Математика,4.0
185,Пермский край,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Математика,7.0
531,Республика Карелия,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1,Математика,4.0
592,Республика Карелия,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,3,Математика,5.0


In [9]:
df['colname'].unique()

array(['Выпуск фактический с 01.10. прошлого года по 30.09. текущего года всего (сумма гр. 10, 12, 13, 15)'],
      dtype=object)